In [4]:
import tensorflow as tf
import numpy as np
import time

tf.reset_default_graph()
tic = time.time()
G = 6.67e5
threshold = 0.1
ts = 1e-4                                                 #time step
M = tf.Variable(np.load("../q2_input/masses.npy"), tf.float64, name="M")        #masses
X = tf.Variable(np.load("../q2_input/positions.npy"), tf.float64, name="X")     #postions
V = tf.Variable(np.load("../q2_input/velocities.npy"), tf.float64, name="V")    #velocities
# A = tf.placeholder(tf.float64, shape=V.shape)             #accelerations
A = tf.Variable(tf.zeros(shape=X.shape), tf.float64, name="A")
chk = tf.Variable(tf.ones(shape=X.shape), tf.float64)
init_op = tf.global_variables_initializer()

# with tf.Session() as :
sess = tf.Session()
sess.run(init_op)

def calcAccl():
    global A
    global chk
    global X
    
    D = sess.run(X)
    D = tf.reshape(D, [1,100,2])
    D = tf.tile(D, [100,1,1])
    
    temp = tf.reshape(sess.run(X), [100,1,2])
    temp = tf.tile(temp, [1,100,1])

    D = tf.math.subtract(D, temp)
    temp = D

    chk = tf.norm(D, axis=2)
    chk = tf.matrix_set_diag(chk, tf.ones([100], dtype=tf.float64))
    
    D = tf.reshape(tf.math.pow(chk, 3*tf.ones([100,100], dtype=tf.float64)), [100,100,1])
#     Mtensor = tf.convert_to_tensor(sess.run(M).reshape(1,100))
#     D = tf.reshape(tf.math.multiply(D, Mtensor), [100,100,1])
    Mtensor = tf.tile(tf.reshape(sess.run(M), [1,100,1]), [1,1,2])
    temp = tf.math.multiply(temp, Mtensor)
    
    D = G * (tf.math.divide(temp, tf.tile(D, [1,1,2])))
    D = tf.math.reduce_sum(D, axis=1)
    
#     toc = time.time()
    A = D
#     print("-toc time- %s seconds ---" % (time.time() - toc))
        
iter = 0
while True:
#     print(sess.run(chk))
    val = tf.math.reduce_min(chk)
    print(sess.run(val))
    if sess.run(val) < threshold:
        #print to files
        print(sess.run(val))
        np.save('finalPos.npy', sess.run(X), allow_pickle = False)
        np.save('finalVel.npy', sess.run(V), allow_pickle = False)
        print("iter = %d" % iter)
        break
    print("iter = %d" % iter)
    calcAccl()
#     X = X + V * ts + 0.5 * A * ts * ts
    X = tf.math.add(X, tf.math.add(V*ts,  A*ts*ts*0.5))
    V = tf.math.add(V, A*ts)
    iter += 1
        
print("--- %s seconds ---" % (time.time() - tic))

# or creating the writer inside the session
writer = tf.summary.FileWriter('./graphs', sess.graph)

1.0
iter = 0
1.0
iter = 1
1.0
iter = 2
1.0
iter = 3
1.0
iter = 4
1.0
iter = 5
0.640594191504327
iter = 6
1.0
iter = 7
0.4767443625173861
iter = 8
1.0
iter = 9
1.0
iter = 10
1.0
iter = 11
1.0
iter = 12
1.0
iter = 13
0.39779469930768174
iter = 14
1.0
iter = 15
1.0
iter = 16
1.0
iter = 17
1.0
iter = 18
1.0
iter = 19
1.0
iter = 20
1.0
iter = 21
1.0
iter = 22
1.0
iter = 23
1.0
iter = 24
1.0
iter = 25
1.0
iter = 26
1.0
iter = 27
1.0
iter = 28
1.0
iter = 29
1.0
iter = 30
0.9084211055748823
iter = 31
1.0
iter = 32
1.0
iter = 33
1.0
iter = 34
1.0
iter = 35
1.0
iter = 36
1.0
iter = 37
1.0
iter = 38
1.0
iter = 39
1.0
iter = 40
1.0
iter = 41
1.0
iter = 42
1.0
iter = 43
1.0
iter = 44
1.0
iter = 45
1.0
iter = 46
1.0
iter = 47
1.0
iter = 48
1.0
iter = 49
1.0
iter = 50
1.0
iter = 51
1.0
iter = 52
0.26998898279058187
iter = 53
0.4502874235131623
iter = 54
0.4135234225459988
iter = 55
1.0
iter = 56
1.0
iter = 57
1.0
iter = 58
0.7976804161237453
iter = 59
0.5128696390816452
iter = 60
0.13950846718157436


In [9]:
import tensorflow as tf
import numpy as np
import time

tic = time.time()
G = 6.67e5
threshold = 0.1
ts = 1e-4                                                 #time step
M = tf.Variable(np.load("../q2_input/masses.npy"), name="M")        #masses
X = tf.Variable(np.load("../q2_input/positions.npy"), name="X")     #postions
V = tf.Variable(np.load("../q2_input/velocities.npy"), name="V")    #velocities
# A = tf.placeholder(tf.float64, shape=V.shape)             #accelerations
A = tf.Variable(tf.zeros(shape=X.shape), tf.float64, name="A")
chk = np.ones((100,100))
init_op = tf.global_variables_initializer()

def calcAccl():
    global A
    global chk
    with tf.Session() as sess:
        sess.run(init_op)
        D = (sess.run(X)).reshape(1, 100, 2)
        D = np.repeat(D, 100, axis=0)

        temp = np.repeat(sess.run(X), 100, axis=0)
        temp = temp.reshape(100,100,2)
        D = D - temp
        temp = D
        
        chk = np.linalg.norm(D, axis=2)
        np.fill_diagonal(chk, 1)
        
        D = np.power(chk, 3)
        D = (D * (sess.run(M).reshape(1,100))).reshape(100, 100, 1)                                      
        D = G * (temp / np.repeat(D, 2, axis=2))
        D = np.sum(D, axis=1)

        A = tf.convert_to_tensor(D, dtype=tf.float64)
        

while True:
    val = chk.min()
#     print(val)
    if val < threshold:
        #print to files
        with tf.Session() as sessi:
            sessi.run(init_op)
            np.save('finalPos.npy', sessi.run(X), allow_pickle = False)
            np.save('finalVel.npy', sessi.run(V), allow_pickle = False)
        break
    calcAccl()
    X = X + V * ts + 0.5 * A * ts * ts
    V = V + A * ts
        
print("--- %s seconds ---" % (time.time() - tic))
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)

--- 85.87928366661072 seconds ---


In [24]:
import tensorflow as tf
import numpy as np
import time

tf.reset_default_graph()
tic = time.time()
G = 6.67e5
threshold = 0.1
ts = 1e-4                                                 #time step
M = tf.Variable(np.load("../q2_input/masses.npy"), tf.float64, name="M")        #masses
X = tf.Variable(np.load("../q2_input/positions.npy"), tf.float64, name="X")     #postions
V = tf.Variable(np.load("../q2_input/velocities.npy"), tf.float64, name="V")    #velocities
# A = tf.placeholder(tf.float64, shape=V.shape)             #accelerations
A = tf.Variable(tf.zeros(shape=X.shape), tf.float64, name="A")
chk = np.ones((100,100))
init_op = tf.global_variables_initializer()

# with tf.Session() as :
sess = tf.Session()
sess.run(init_op)

def calcAccl():
    global A
    global chk
    global X
    
#     sess.run(init_op)
    D = (sess.run(X)).reshape(1, 100, 2)
    D = np.repeat(D, 100, axis=0)

    temp = np.repeat(sess.run(X), 100, axis=0)
    temp = temp.reshape(100,100,2)
    D = D - temp
    temp = D

    chk = np.linalg.norm(D, axis=2)
    np.fill_diagonal(chk, 1)

    D = np.power(chk, 3)
    D = (D * (sess.run(M).reshape(1,100))).reshape(100, 100, 1)                                      
    D = G * (temp / np.repeat(D, 2, axis=2))
    D = np.sum(D, axis=1)
    
#     toc = time.time()
    A = tf.convert_to_tensor(D, dtype=tf.float64)
#     print("-toc time- %s seconds ---" % (time.time() - toc))
        
# iter = 0
while True:
    val = chk.min()
#     print(val)
    if val < threshold:
        #print to files
        np.save('finalPos.npy', sess.run(X), allow_pickle = False)
        np.save('finalVel.npy', sess.run(V), allow_pickle = False)
        print("iter = %d" % iter)
        break
    calcAccl()
#     X = X + V * ts + 0.5 * A * ts * ts
    X = tf.math.add(X, tf.math.add(V*ts,  A*ts*ts*0.5))
    V = tf.math.add(V, A*ts)
#     iter += 1
        
print("--- %s seconds ---" % (time.time() - tic))

# or creating the writer inside the session
writer = tf.summary.FileWriter('./graphs', sess.graph)

iter = 1
--- 26.891173839569092 seconds ---
